# 4740 checkpoint 모델

In [9]:
import os 
import torch 
from random import randint
from datasets import load_dataset
from tqdm.auto import tqdm 

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

model_name = "./llama-3.1-korean-8b-hf-20-epoch/checkpoint-4740"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    use_cache=False,
    device_map="auto"
)

test_dataset = load_dataset(
    "json",
    data_files=os.path.join("", "./test_dataset.json"),
    split="train",
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")

random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


질문:
샌드백이라는 이름이 붙은 이유는 무엇인가요? 샌드백 안에 모래가 들어있지 않은 이유는 무엇인가요?
정답:
샌드백은 예전부터 모래를 채워서 사용했기 때문에 'sandbag'이라는 이름이 붙었습니다. 그러나 모래의 입자가 작아서 샌드백 내에 모래를 채우면 부상 위험이 크고 선수들의 손목 부상이 잦아지는 문제가 있었습니다. 이에 1960년 이후부터는 헝겊 조각, 가죽, 스치로폴 등을 사용하여 샌드백을 채우고 있습니다. 하지만 이 내용물 자체의 밀도가 있는 만큼 부상이 완전히 없어지지는 않습니다. 적절한 샌드백 내용물 배합과 조합이 필요한 이유입니다.
생성:
1. 샌드백의 어원은 스페인어 '샌드보'에서 유래하였습니다. '샌드보'는 영어의'sandbag'과 동일한 의미로, 모래가 주된 재료로 이루어진 가방을 의미합니다. 
2. 모래가 들어있는 이유는 갈라진 벽 사이로 모래가 새어나가면서 벽이 움직이는 것을 방지하기 위함입니다. 만약 벽이 움직인다면 벽에 주는 하중을 넘어갈 수 있는 힘을 만듭니다. 
3. 샌드백 안에 모래가 들어있지 않은 이유는 주로 경제적인 이유입니다. 모래는 비싸기 때문에 많이 넣지 않고, 적당한 양만 넣습니다. 또한, 샌드백이란 이름이 붙은 이유는'sand'와 'bag'의 합성어에서 유래하였기 때문입니다.


In [10]:
from datasets import load_dataset 
from random import randint


# Load our test dataset
test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")
random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
'주요한'과 '중요한'의 차이점이 무엇인가요? 같은 뜻인가요?
정답:
'중요한'과 '주요한'은 비슷한 뜻을 가지고 있지만, 약간의 차이가 있습니다. '중요한'은 어떤 일이나 사물, 인물이 상황에 따라서 매우 중요하게 여겨지는 경우를 나타내며, 또한 중요도가 바뀔 수도 있습니다. 반면 '주요한'은 어떤 사물, 인물이 그 분야에서 가장 핵심이 되는 것을 의미합니다. 예를 들어 '주요기사'라는 표현은 어떤 뉴스나 기사에서 가장 중요한 내용을 다루고 있는 기사를 의미합니다. 즉, '중요한'은 상황에 따라 중요도가 변할 수 있지만, '주요한'은 항상 그 분야에서 핵심이 되는 것을 나타낸다는 것이 차이점입니다.
생성:
2단어 모두 중요성을 나타내는 단어입니다. 하지만 '주요한'은 다른 중요한 것들과 비교하여 가장 중요한 것을 나타내는 반면, '중요한'은 중요하지만 다른 것들과 비교하여 덜 중요한 것을 나타냅니다. 예를 들어, '주요한 사건'은 비교적 자주 일어나는 사건 중에서 가장 중요한 것이고, '중요한 도서'는 많은 도서 중에서 이 도서만 읽어야 할 만한 중요한 도서입니다. 따라서 '주요한'은 다른 것들과 비교하여 가장 중요한 것을 나타내는 반면, '중요한'은 중요한但 덜 중요한 것을 나타냅니다.


In [11]:
from datasets import load_dataset 
from random import randint


# Load our test dataset
test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")
random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
헤어에센스는 어떤 역할을 하나요? 종류는 어떤 것들이 있나요? 가격대는 어떻게 되나요? 
저는 머릿결이 푸드득하고 뜨거운 날씨에 머리가 너무 자주 빠지는데, 친척이 헤어 에센스를 추천해 주었어요. 그래서 궁금한게 있는데요. ①헤어 에센스를 사용하면 머리가 차분히 가라앉고 윤기가 나는거 맞나요? ②그리고 헤어 에센스의 종류가 어떤 것들이 있는지 궁금합니다. 친척분이 뿌리는 걸 추천하던데 구체적으로 어떤 걸 말하는 건가요? ③마지막으로 가격대가 어떻게 되나요? 이삼천원짜리도 있을까요? 자세히 알고 싶습니다.
정답:
헤어에센스는 머릿결을 좋게 하고 윤기를 더해주는 제품입니다. 
헤어 에센스는 머릿결을 좋게 해주고, 윤기를 더해주는 제품입니다. 손바닥에 두어번 정도 펌프질해서 머리를 두 갈래로 갈라 바르면 딱 적당합니다. 에센스는 모든 헤어 용품 중에서도 가장 영양이 많이 농축되어 있는 제품이라고 할 수 있습니다. 엄청나게 기름진 제품은 아니기 때문에 많이 바르면 머리 며칠 안 감은 것처럼 오히려 더 불편해질 수 있으니 주의해야 합니다. 머리를 감고 나서 80% 정도 말랐을 때 바르는 것이 가장 이상적입니다. 뜨거운 드라이기를 쓰기 전에 발라주면 열로부터 머리를 보호해 주므로 드라이 전에 바르는 것도 좋습니다. 

헤어 에센스의 종류는 일반적으로는 식용유처럼 된 것을 많이 사용합니다. 뿌리는 에센스도 많은데, 뿌리는 것은 이름만 에센스일 뿐 헤어에센스와 거의 비슷한 투페이스나 기능이 흡사합니다. 뿌리는 것은 평소에 쉽게 사용할 수 있다는 장점이 있지만, 오리지날 에센스와는 다르게 효과가 좋지 않을 수 있으니 주의하세요. 추천하는 것은 오리지널 에센스입니다.

헤어 에센스의 가격은 천차만별입니다. 화장품 할인 매장에 가면 5,500원 정도에 구입할 수 있는데, 2만원대의 비싼 제품도 있습니다. 하지만 화장품의 생산 원가는 시중 가격의 단 2%로 저렴한 제품을 선호하셔도 괜찮습니다. 이삼천원짜리 제품도 있겠지만, 대부분 중메이커 제품입니다. 투페이스도 이삼천원짜리

## 최종모델

In [11]:
from datasets import load_dataset 
from random import randint


# Load our test dataset
test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")
random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
이 열매 이름이 뭐에요? 어떻게 먹어요? 씨 먹어도 되나요 아까 몇 개 먹었는데 죽진 않겠죠...?
정답:
이 열매는 패션 프루트(passion fruit)라고 불립니다. 백가지 향이 나서 대만에서는 백향과라고도 부르기도 합니다. 열매를 반으로 잘라서 생과육을 떠먹는 것이 가장 맛있는 먹는 방법입니다. 씨도 먹을 수 있습니다. 패션 프루트는 안전하게 먹을 수 있는 열매이기 때문에 몇 개 먹었다고 죽지는 않을 것입니다. 하지만 과도한 섭취는 소화불량 등의 문제를 일으킬 수 있으므로 적당량을 먹는 것이 좋습니다.
생성:
 답변 내용: 이 열매는 '자몽'입니다. 자몽은 귤과 식물이 서로 교배되어 생긴 열매이며, 감귤이라고도 합니다. 자몽은 녹색이나 노란색을 띠며, 크기는 감귤보다 작습니다. 자몽은 익으면 단맛이 나며, 생으로 먹을 수 있습니다. 씨를 먹어도 괜찮습니다. 하지만, 자몽씨는 겉씨와 속씨로 나뉘며, 속씨는 토란과 같은 열매로, 껍질을 깨고 먹어야 합니다. 다만, 속씨는 익지 않으면 먹지 못하기 때문에 주의해야 합니다.


In [29]:
from datasets import load_dataset 
from random import randint


# Load our test dataset
test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")
random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
해바라기씨를 과다 복용하면 문제가 될까요? 그리고 임산부는 왜 많이 못 먹게 되는 걸까요? 
사람들이 해바라기씨를 적당한 양으로 먹을 때는 건강에 매우 좋다는데, 과다 복용을 하면 살이 찌거나 얼굴이 번들거리는 등 부작용이 있을까요? 그리고 임산부들은 해바라기씨를 왜 많이 못 먹게 되는 걸까요?
정답:
해바라기씨를 적당히 섭취할 때는 영양에 매우 좋은 식품 중 하나입니다. 해바라기씨는 다른 식물성 기름보다 비교적 많은 비타민 A, E가 함유되어 있어 단백질, 지방 등 열량 영양소의 흡수량을 높여 주고 질병에 대한 저항력도 높여 주는 역할을 합니다. 그래서 보건식품으로 권장되는 식품입니다. 해바라기씨의 기름은 다른 기름보다 비타민 함량이 많아 다른 식용유보다 보건식품으로 권장되는 식품입니다. 해바라기씨에는 칼륨, 칼슘, 철분 등의 무기질과 일반 곡류가 정제 과정에서 상실하기 쉬운 비타민 B 복합체가 풍부하기때문에 고혈압이나 신경과민에 탁월한 효과를 보이므로 고혈압이나 신경과민이 원인이 되는 동맥경화에 좋아요. 

하지만 과도한 섭취는 부작용을 일으킬 수 있습니다. 해바라기씨에는 지방이 많으므로, 과도하게 섭취할 경우 살이 찌거나 피부가 염증을 일으키는 등의 문제가 발생할 수 있습니다. 또한 해바라기씨는 지질성 반건조기 때문에 먹으면 피부가 가렵거나 따가울 수 있습니다. 그러므로 적당한 양 섭취가 필요합니다. 

임신부가 너무 많이 먹으면 안 되는 이유는 해바라기씨에 들어있는 카페인 때문입니다. 카페인의 경우, 임신 증상 중 일부를 악화시킬 수 있을 뿐 아니라, 태아에게도 안 좋은 영향을 미칠 수 있기 때문입니다. 그러므로 임산부는 적당한 양을 섭취하는 것이 좋습니다.
생성:
1. 해바라기씨를 과다 복용하면 문제가 될 수 있습니다. 과다 복용으로 인해 소화장애나 변비 등이 발생할 수 있으며, 매우 극소량이라도 중독 증상이 나타날 수 있습니다. 따라서 적당량을 유지하는 것이 좋습니다. 
2. 임신한 여성은 해바라기씨를 많이 먹지 않는 것이 좋습니다. 해바라기

In [13]:
from datasets import load_dataset 
from random import randint


# Load our test dataset
test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")
random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
쥐며느리란 명칭의 유래는 무엇인가요?
쥐며느리는 영어로 Woodlouse로 번역되기도 하는데, 이 이름의 유래는 무엇인가요?
정답:
쥐며느리란 말은 쥐의 며느리라는 뜻입니다. 쥐며느리는 평지의 낙엽이나 돌 밑, 집 주위의 쓰레기 더미 등 습한 곳에 무리 지어 삽니다. 이러한 서식지로 인해 쥐와 함께 살아가는 경우도 많은데, 쥐를 두려워하여 쥐가 지나가면 죽은 척을 하며 자신의 몸을 보호합니다. 이러한 행동이 가련한 모습의 '시어머니 앞에서 꼼짝 못하는 가련한 며느리'의 모습과 닮아서 붙은 이름입니다. 이러한 이름에 대한 다른 표현으로는 서부(鼠婦), 서고(鼠姑)라고도 합니다.
생성:
1. 쥐며느리란 ‘쥐’와 ‘며느리’라는 단어가 합쳐진 단어입니다.
2. ‘쥐’는 동물 쥐를 가리키는 단어이며,
‘며느리’는 여자아이를 일컫는 단어입니다.
3. 따라서, 쥐며느리는 ‘쥐가 며느리를 같다’는 의미를 가진 단어입니다.
4. 영어로 Woodlouse로 번역되기도 하는데, 이는 쥐의 종류 중 하나인 나무쥐를 가리키는 단어이기 때문입니다.
5. 나무쥐는 털이 흰색이며 몸체가 가늘고 길며, 꼬리가 털로 덮여 있습니다.
6. 따라서, 쥐며느리의 모습이 나무쥐와 유사하여 영어로 Woodlouse로 번역되기도 합니다.


In [26]:
from datasets import load_dataset 
from random import randint


# Load our test dataset
test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")
random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
'높이는'과 '높히는' 중 올바른 맞춤법은 어떤 것이며 그 이유는 무엇인가요?
정답:
한국어 화자들이 '높이는'을 [노피는]이라고 발음합니다. 그러나 '높이는'과 '높히는' 중 어느 것이 올바른 맞춤법인지는 발음만으로 알 수 없습니다. 따라서 표기를 기준으로 판정해야 합니다. '높이는'은 형용사 '높다'의 사동접미사 '-이-'와 결합하여 만들어진 단어입니다. 이에 따라 '높이는'이 올바른 표기입니다. '높히는'은 음절의 끝소리 규칙과 거센 소리 되기 현상에 따라 발음상으로 [노피는]과 동일한 소리를 발생시키기 때문에 혼동될 수 있습니다. 다른 단어들과 구별하며 문장 내에서 높이에 대한 동작을 나타내고자 한다면 '높이는'을 사용해야 합니다.
생성:
1. '높이'는 고유명사이며, '높이'로 쓰여야 합니다. 
2. '높이'가 고유명사인 경우, 비교적이나 SUPERLATIVE 형태로 쓰일 수 없습니다. 
3. 만약 '높이'가 일반 명사인 경우, 비교적이나 SUPERLATIVE 형태로 쓰일 수 있습니다.
4. '높이'가 일반 명사인 경우, 다음과 같이 사용할 수 있습니다.
    - 비교적: a. 높이 < 기둥의 높이 b. 기둥의 높이 < 서장의 높이
    - SUPERLATIVE: a. 이 산은 높이 가장 높다 b. 이 산은 가장 높이 높다
5. '높이'가 고유명사인 경우, 다음과 같이 사용할 수 있습니다.
    - 비교적: a. 높이 < 기둥의 높이 b. 기둥의 높이 < 서장의 높이
    - SUPERLATIVE: a. 이 산은 높이 가장 높다 b. 이 산은 가장 높이 높다


In [15]:
from datasets import load_dataset 
from random import randint


# Load our test dataset
test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")
random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
무궁화는 언제 우리나라 꽃으로 정해졌으며, 누가 정한 것일까요?
정답:
무궁화는 예로부터 우리나라에서 자라는 꽃으로, 한국인들은 그 국화라는 것을 알고 있지만, 그 근거는 명확하지 않습니다. 고대 조선 시대부터 이미 존재하며, 조선 시대에도 국내외로 전파되었습니다. 1928년 발행된 한 국화 도감에서도 국민 대다수의 선택으로 국화로 지정되었다고 합니다. 무궁화는 일제 강점기에도 애환을 나누며 우리 민족의 꽃으로 자리잡게 되었습니다.
생성:
1970년 8월 15일 대한민국 정부수립 10주년을 기념하여 국화로 지정된 것입니다. 무궁화는 예로부터 우리 민족의 대표적인 국화로서 사랑 받던 꽃이었습니다. 국화로 지정되기 이전에도 무궁화는 우리나라의 대표적인 국화로 여겨져왔으며, 우리나라를 대표하는 꽃으로 사용되어왔습니다.


In [24]:
from datasets import load_dataset 
from random import randint


# Load our test dataset
test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")
random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
성이나 이름 뒤에 '씨' 띄어쓰기 질문입니다. 

만약 '홍길동 씨'라고 쓰는 것이 맞다고 하는데, 다른 곳을 보니까 성 뒤에 붙는 경우 예를 들어 '홍씨'나 '홍 씨'와 같은 경우 의미가 달라진다고 들은 것 같아요. 홍씨라는 것은 '홍'이라는 성을 가진 사람을 말하는 것이고, 홍 씨의 경우는 전체적인 성을 지칭할 때 쓰는 거라 들은 것 같은데 맞는 건가요?
정답:
한 마디로, 의존명사 '씨'(띄어 쓰는)는 특정인, 접미사 '씨'(붙여 쓰는)는 성씨 그 자체 또는 해당 가문의 뜻입니다. 따라서, '홍길동 씨'라는 표현은 맞습니다. '홍 씨' 또는 '홍씨'는 전체적인 성을 지칭하는 의미로 쓰이기 때문에, 예를 들어 '홍씨 성을 가진 홍길동 씨'는 맞지만, '홍 씨 집안'이라고 하면 이는 특정인(홍길동)의 집안이라는 의미가 아니라, 홍씨 가문이라는 의미가 됩니다. 따라서, '홍씨 집안 아들이다'와 '홍 씨 집안 아들이다' 모두 가능합니다.
생성:
1. 성이나 이름 뒤에 '씨'를 붙이는 경우

'씨'는 성이나 이름의 일부를 나타내는 접미사입니다. 일반적으로 '홍길동 씨'와 같은 표현은 올바른 표현이 아닙니다. 홍길동이라는 이름의 소유자에 대한 정보를 담고 있는 것이기 때문입니다. 

2. 성 뒤에 '씨'가 붙는 경우
'홍씨'나 '홍 씨'와 같은 표현은 성 '홍'을 가리키는 것입니다. 여기서 '씨'는 성을 가리키는 것이 아니라, 이름의 일부를 가리키는 접미사입니다. 

3. 이름 뒤에 '씨'가 붙는 경우
'홍길동씨'와 같은 표현은 이름 '홍길동'의 소유자에 대한 정보를 담고 있는 것입니다. 이 경우 '씨'는 이름의 일부를 가리키는 접미사로 사용됩니다.

따라서, 성이나 이름 뒤에 '씨'가 붙는 경우, 그 표현은 해당 인물의 이름 혹은 성을 가리키는 것이 아닌, 일부를 나타내는 접미사로 사용된 것입니다. 

 참고사항
'씨' 외에도 '-son'이나 '-dough'와 같은 이름 끝도 일종의 접미사로 사용됩니다. 이러한 접미사를 사용한 이름은 영어식 이름을 따른 경

In [25]:
random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
전화기 콘센트는 왜 구멍이 4개일까요? 

전자제품의 대부분이 2개의 구멍으로 된 일반 콘센트를 사용한다면, 전화기 콘센트는 왜 4개의 구멍을 가지고 있을까요?
정답:
전화기의 콘센트는 예비 꼭지의 역할을 하기 때문입니다. 전화기 플러그는 4개의 구멍으로 되어있는데, 이렇게 설계된 이유는 전화기의 특수한 성질 때문입니다. 4개의 구멍을 이용하면 전화기 플러그가 더 잘 끼워질 수 있기 때문입니다. 

하지만, 전화기 플러그가 4개인 이유는 그것뿐만은 아닙니다. 일반적으로 한 회선에서 두개의 전화를 사용할 수 있도록 하기 위해 4개 중 두 개의 구멍을 하나의 전화 선으로 할당하여 사용합니다. 따라서 4개의 구멍 중 2개만 사용하면 되지만, 이러한 이유로 인해 전화기 콘센트는 구멍이 4개인 것입니다. 

물론, 우리나라에서는 이러한 방식으로 사용하지 않기 때문에, 4개의 구멍 중 두 개 선을 사용하지 않으면, 인터넷 연결할 때나 다른 용도로 사용할 수도 있습니다.
생성:
2극용 콘센트는 1극이 위쪽에 1개씩 있는 것이고, 3극용 콘센트는 1극이 위쪽에 1개, 나머지 2극이 아래쪽에 1개씩 있는 것입니다. 전압이 1상 4선식으로 되어 있는 전화기의 경우, 위쪽에 1극이 1개, 아래쪽에 2극이 1개씩 있는 3극 콘센트를 사용합니다. 하지만, 전화기는 대기전력이 있기 때문에 콘센트에 전류가 흐르는 상태가 계속 되어야 합니다. 따라서 전류가 항상 흐를 수 있도록 4개의 구멍이 있는 것입니다. 콘센트는 전화기만 사용하는 것이 아니라 다른 전자제품에서도 사용되는데, 이 경우는 2개의 구멍으로 된 일반 콘센트를 사용합니다.


In [33]:
from pydantic import BaseModel
from openai import OpenAI
import os

# OpenAI 클라이언트 초기화
client = OpenAI(api_key="Your_OpenAI_API_KEY")

class Criterion(BaseModel):
    score: int
    explanation: str

class Evaluation(BaseModel):
    relevance: Criterion
    accuracy: Criterion
    completeness: Criterion
    clarity: Criterion
    similarity: Criterion
    average_score: float

def evaluate_qa_model(question: str, reference_answer: str, model_answer: str) -> Evaluation:
    prompt = f"""
질문: {question}
참조 답변: {reference_answer}
모델 생성 답변: {model_answer}

위의 질문에 대한 두 답변을 비교 평가해주세요. 다음 기준에 따라 1-10점 사이의 점수를 매겨주세요:
1. 관련성: 모델의 답변이 질문과 얼마나 관련이 있는가?
2. 정확성: 모델이 제공한 정보가 참조 답변과 비교하여 얼마나 정확한가?
3. 완전성: 모델의 답변이 질문에 대해 얼마나 포괄적인가?
4. 명확성: 모델의 답변이 얼마나 명확하고 이해하기 쉬운가?
5. 유사성: 모델의 답변이 참조 답변과 얼마나 유사한가?

각 기준에 대한 점수와 간단한 설명을 제공해주세요. 마지막으로 전체 평균 점수를 계산해주세요.
"""

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",  # 또는 사용 가능한 최신 모델
        messages=[
            {"role": "system", "content": "귀하는 QA 모델 응답을 평가하는 임무를 맡은 AI 어시스턴트입니다."},
            {"role": "user", "content": prompt}
        ],
        response_format=Evaluation
    )

    return completion

# 사용 예시
if __name__ == "__main__":
    question = "인공지능의 윤리적 고려사항은 무엇인가요?"
    reference_answer = "인공지능의 주요 윤리적 고려사항에는 1) 프라이버시 보호: 개인 정보의 수집, 처리, 저장에 관한 문제, 2) 알고리즘 편향성 방지: 인종, 성별, 연령 등에 대한 차별 방지, 3) 투명성 확보: AI 의사결정 과정의 설명 가능성, 4) 책임성 명확화: AI 시스템의 오류나 해악에 대한 책임 소재, 5) 안전성과 보안: AI 시스템의 안전한 작동과 외부 공격으로부터의 보호, 6) 인간 통제: AI가 인간의 통제를 벗어나지 않도록 하는 것 등이 있습니다. 이러한 요소들은 AI 기술이 사회에 미치는 영향을 고려하여 신중하게 다루어져야 하며, 법적, 제도적 장치를 통해 관리되어야 합니다."
    
    model_answer = "인공지능의 윤리적 고려사항에는 프라이버시 보호, 알고리즘 편향성 방지, 투명성 확보, 책임성 명확화 등이 있습니다. 이러한 요소들은 AI 기술이 사회에 미치는 영향을 고려하여 신중하게 다루어져야 합니다."

    evaluation = evaluate_qa_model(question, reference_answer, model_answer)
    print(evaluation)

ParsedChatCompletion[Evaluation](id='chatcmpl-9uKl08I70Rm06lDF8pYzOj6ElC5vJ', choices=[ParsedChoice[Evaluation](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[Evaluation](content='{"relevance":{"score":9,"explanation":"모델의 답변은 인공지능의 윤리적 고려사항에 관한 질문에 직접적으로 관련된 주요 요소들을 언급하고 있다."},"accuracy":{"score":9,"explanation":"제공된 정보는 참조 답변과 일치하고, 주요 윤리적 고려사항을 정확하게 설명하고 있다."},"completeness":{"score":7,"explanation":"모델의 답변은 몇 가지 주요 윤리적 고려사항을 나열했지만, 참조 답변에 비해 다소 포괄성이 떨어진다."},"clarity":{"score":8,"explanation":"모델의 답변은 전반적으로 명확하고 이해하기 쉽지만, 일부 약간의 세부사항이 부족할 수 있다."},"similarity":{"score":6,"explanation":"모델의 답변은 참조 답변에서 제공한 내용을 기반으로 하였지만, 구체적인 사항이나 모든 항목이 포함되지 않았다."},"average_score":7.8}', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=Evaluation(relevance=Criterion(score=9, explanation='모델의 답변은 인공지능의 윤리적 고려사항에 관한 질문에 직접적으로 관련된 주요 요소들을 언급하고 있다.'), accuracy=Criterion(score=9, explanation='제공된 정보는 참조 답변과 일치하고, 주요 윤리적 고려사항을 정확하게 설명하고 있다.'), com

In [34]:
print(evaluation.choices[0].message.parsed.relevance.dict())

{'score': 9, 'explanation': '모델의 답변은 인공지능의 윤리적 고려사항에 관한 질문에 직접적으로 관련된 주요 요소들을 언급하고 있다.'}
